<a href="https://colab.research.google.com/github/litongzheng/APS360Project/blob/master/aps360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [42]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [43]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
# sp = pd.read_csv("All-seasons.csv")
# sp.replace('\n',' ',regex = True,inplace=True)
# sp.to_csv('sp.csv',index= False)
# sp.head()
# cartman = sp[sp.Character=="Cartman"]
# stan = sp[sp.Character == "Stan"]

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
sp_text = ""

chars = []
i = 0
for char in open('/content/drive/My Drive/Colab Notebooks/character.txt'):
  chars.append(char.strip() + ":\t")
  if i>=70000:
    break
  i = i+1
lines = [] 

i = 0
for line in open('/content/drive/My Drive/Colab Notebooks/Line.txt',encoding="ISO-8859-1"):
  lines.append(line)
  if i>=70000:
    break
  i = i+1
        
for i in range(len(lines)):
  total = chars[i] + lines[i]
  sp_text += total


In [27]:
len(sp_text)

5054277

In [28]:
vocab = list(set(sp_text))
vocab_stoi = {s: i for i, s in enumerate(vocab)}
vocab_itos = {i: s for i, s in enumerate(vocab)}
len(vocab)

131

In [30]:
import random
random.seed(10)

sp_len = len(sp_text)

def random_chunk(chunk_len=300):
    """Return a random subsequence from `spam_text`"""
    start_index = random.randint(0, sp_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return sp_text[start_index:end_index]

print(random_chunk())

an:	You see Token, people really enjoy seeing African-Americans on the news... Seeing African-Americans on the news, not hearing them. That's why all African-Americans newspeople learn to talk more... wha, how should I say... white.  Token, all the great African-Americans newspeople have learned to h


In [31]:
def text_to_tensor(text, vocab=vocab):
    """Return a tensor containing the indices of characters in `text`."""
    indices = [vocab_stoi[ch] for ch in text]
    return torch.tensor(indices)

print(text_to_tensor(random_chunk()))


tensor([ 56,  54,  49,  10, 118,  96,  10, 115,  46,  28,  18, 109,  67,  10,
         19,  74,  92,  38,  10,  38, 109,  19, 110, 130, 118, 109,  12,  96,
         10,  89,  10, 109,  74,  93,  49,  67, 109,  93, 112, 109,  75,  45,
         37,  19,  92,  89,  45,  93,  92,  67, 109,  75,  93,   4,  67, 109,
        110,  89,  10, 109, 110,  74,  74, 109, 118,  92,  49, 109,  49,  93,
         44,  10,  49,  96,  10,  89,  73, 109,  79,  96, 110,  49, 109,  67,
         93,  92, 115,  38,  67, 109,  74,  45,  33,  10, 109, 110, 109,  44,
         93,  93,  38, 109,  45,  38,  10, 110,  47, 109,  56,  62,  92,  49,
         49,  10,  89,  67,  46,  28,  42,  96,  93, 110, 109,  75,  93,   4,
         86, 109, 117, 110, 130, 118,  86, 109,  56, 117, 110,  89,  49, 130,
        110, 115,  46,  28,  65, 110, 110,  96, 110, 110,  96, 110,  96, 110,
         96, 110,  86, 109, 109, 113,  10,  89,   4, 109, 112,  92, 115, 115,
          4,  86, 109,  30, 109,  67,  92, 118, 118,  93,  67,  

In [0]:
def random_training_set(chunk_len=1000):    
    chunk = random_chunk(chunk_len)
    inp = text_to_tensor(chunk[:-1])   # omit the last token
    target = text_to_tensor(chunk[1:]) # omit the first token
    return inp, target

In [0]:
class Cartmanboi(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(Cartmanboi, self).__init__()
        # RNN attributes
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        # identiy matrix for generating one-hot vectors
        self.ident = torch.eye(vocab_size)
        # recurrent neural network
        self.rnn = nn.GRU(vocab_size, hidden_size, n_layers, batch_first=True)
        # a fully-connect layer that decodes the RNN output to
        # a distribution over the vocabulary
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inp, hidden):
        # reshape the input tensor to [1, seq_length]
        inp = inp.view(1, -1)
        # generate one-hot vectors from token indices
        inp = self.ident[inp]
        # obtain the next output and hidden state
        output, hidden = self.rnn(inp, hidden)
        output = self.decoder(output.squeeze(0))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

In [0]:
def evaluate(model, prime_str='I', predict_len=300, temperature=0.8):
    hidden = model.init_hidden()
    prime_input = text_to_tensor(prime_str)
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        predicted += predicted_char
        inp = text_to_tensor(predicted_char)

    return predicted

def train(model, num_iters=2000, lr=0.004):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for it in range(num_iters):
        # get training set
        inp, target = random_training_set()
        # cleanup
        optimizer.zero_grad()
        # forward pass
        hidden = model.init_hidden()
        output, _ = model(inp, hidden)
        loss = criterion(output, target)
        # backward pass
        loss.backward()
        optimizer.step()

        if it % 200 == 199:
          print("[Iter %d] Loss %f" % (it+1, float(loss)))
          print("    " + evaluate(model, ' ', 300))


In [45]:
model = Cartmanboi(len(vocab),256,2)
train(model, num_iters=2000,lr=0.004)

[Iter 200] Loss 2.194304
     hiy an gols suy bige you wan the tout I's redid, ut inos its, tou'd is gou dout're tise node to  Gatte tou on dian is tave merte he ke mome just be beon'w attid ing! 
Man:	Oh way! 
Kangn:	I't wis, no gorarim wat, in the,  at Lad you it weot ke:	Dow beed le sap sit stor met, wit's of whack mere angis
[Iter 400] Loss 1.888162
     the gorkn for dayd Emat to I nevince Cartmana:	Mare!  
Butter:	How. Elle, Chring tual and home huve to kilk on to bet far watte to go cuvene pallist me hive bever the nees on aver cantery for it's kith can to spreatigro junte'll rightive cally parteve thin't man rong ulay fell the beat certmallyof m
[Iter 600] Loss 2.039475
     I ciple the clamp and theres a wish werin's a nay snuther shad at and me they ale not mand an and youh and stever mund there monet regry how. 
Butters:	No, where the ging they bo mike aptries gotpbackey a like that not have the poenther durn stars five the sarmang suppial they're and fuckar fat to m
[Iter 8

In [46]:
train(model, num_iters=3000,lr=0.004)

[Iter 200] Loss 1.579565
     stucking all be comice that me! 
Stan:	Well you love you could play on there bragett who handy me feel call back to thif ling muse good better mean! 
Dr. Terrince:	Futter missive a cal doing cholatus. 
Terrancon:	Oh mying you will a boytel. 
Cartman:	Does that my one starrit? me to lest on the dils 
[Iter 400] Loss 1.905129
     pyoppore!åÊ 
Guy:	What from the right of us ack. I don if you sume years of you.  
Butters:	Cartman it.  
Randy:	Whooo about a deel Chaigh it.  A sweet years? 
Kyle:	Uh, what can she doing everyone for your Dindror Randy life it? 
Cartman:	But you can free think and I supet a big and like the sure, 
[Iter 600] Loss 1.893726
     for the Indignaffay right well, but why deset! 
Cartman:	I got the needer for answer muney!  He's cots cool, I ass me. 
Cartman:	Oh my God!åÊåÊIt is of comnamen at me to thinkien in the frog ser who's like of my fise's who was out of tran. 
Cartman:	You got sure Kyle lame. A will master and ever onl
[Iter 8

In [112]:
a=evaluate(model, prime_str='Cartman:', predict_len=300, temperature=0.8)
print(a)

Cartman:	Yeah! You prodelam's parents ore was can say too about the streets. 
Butters:	Help gentlemen! Greets!  Oh, Mar, you guy? 
Dr Exeger Hindsod Bom frog feels just aren't Momacher!  
Cartman:	Help me a job.  
Cartman:	Dam, you really fuck a courthous to me, can't be week so we go watch me, real here! 



In [113]:
train(model, num_iters=5000,lr=0.004)

[Iter 200] Loss 1.325661
     like to your frick man... that we weld not anybody think. 
Scouts:	Yeah, too.  
Cartman:	Bear Kenny of you know what the time play stupid unterman with the XBody Damning human to the time tine my should be gonna stupid us. 
Cartman:	That's it of son. 
Jesus:	Maybe it's a fat. 
Leave God:	We, uh, dud
[Iter 400] Loss 1.087876
     the batts we can do that us farmach! Now can try it. 
Representy:	Look trouther Broflovski, sick and just down to me.  
Cartman:	I agn already be money! That me away Aiden. We just just true, who's there! 
Cartman:	Just startly sent time! 
Cartman:	Oh, you guys! The Fight is tired to get to at the g
[Iter 600] Loss 1.357392
     idea. 
Cartman:	Well, there's compleantly, Fills. You're those dollars. 
Stan:	Plan, but I don't know what it's a many some. Somebody are gonna have to stant a kays it's a chickler is gonna get me?! 
Cartman:	Szaul. 
Butters:	Have it was this lister gay with I againit to an and Phillip is can feel b
[Iter 8

In [136]:
a=evaluate(model, prime_str='Kyle:', predict_len=300, temperature=0.8)
print(a)

Kyle:	Well, I know, Wendy is the clud always sent to be a mountain around to go spend next to anything last that don't make a soon of along for that so much.  
Member 2:	Well you guys care on your uncleta I'm not a boy is cool. The way turn for a whole how doesn't happer out.  
Cartman:	What are you doin
